In [ ]:
# PyCalphad (Thermodynamics Calculations & Plotting)
from pycalphad import Database, calculate, equilibrium, variables as v, Model, ternplot
from pycalphad.plot.utils import phase_legend

import numpy as np
import sympy as sp
from sympy import symbols, diff
import matplotlib.pyplot as plt

from itertools import combinations
from collections import defaultdict

In [ ]:
# Load database and choose the phases that will be plotted
db = Database(r'../TDDatabaseFiles_temp/alfe.tdb')

phases = list(db.phases.keys())
constituents = list(db.elements)
legend_handles, color_dict = phase_legend(phases)

# Remove the sublattice phases
for phase in ['AL13FE4', 'AL5FE2', 'B2_BCC', 'AL2FE']:
    if phase in phases:
        phases.remove(phase)

print(phases)
print(constituents)

In [ ]:
def fit_poly(x_data, y_data):
    # Construct the design matrix
    A = np.column_stack([
        np.ones_like(x_data), 
        x_data,               
        x_data**2       
    ])
    
    # Solve the least squares problem to get the coefficients
    coeffs, residuals, rank, s = np.linalg.lstsq(A, y_data, rcond=None)
    
    # Create sympy symbols for x and y
    x= sp.symbols('x')
    
    # Define the list of polynomial terms in the same order as in A:
    terms = [
        1,         
        x, 
        x**2
    ]
    
    # Build the polynomial expression by summing coeff * term for each term.
    expr = sum(sp.Float(coeff) * term for coeff, term in zip(coeffs, terms))
    
    return sp.simplify(expr), residuals

In [ ]:
# Precompute the energy functions for the phases at a temperature
temp = 400
phase_poly_dict = dict()
for phase in phases:
    result = calculate(db, constituents, phase, T=temp, P=101325, output='HM') # Constant pressure
    x_data = result.X.sel(component='FE').values.ravel()
    y_data = result.HM.values.ravel()

    poly, res = fit_poly(x_data, y_data)
    phase_poly_dict[phase] = poly

In [ ]:
def quad_discriminant(expr, var):
    # Extract coefficients of x^2, x, and constant term
    collection = sp.collect(expr, var)
    a = collection.coeff(var, 2)  # Coefficient of x^2
    b = collection.coeff(var, 1)  # Coefficient of x
    c = collection.coeff(var, 0)  # Constant term

    # Compute the discriminant manually
    d = b**2 - 4*a*c

    return d

def projection_function(f1, f2):
    '''
    Computes the projection of f2 onto f1.
    
    Parameters:
        f1 : sympy expression

        f2 : sympy expression
    
    Returns:
        proj : sympy expression

        variables : tuple

        pvariables : tuple
    '''
    free_symbols_f1 = f1.free_symbols
    free_symbols_f2 = f2.free_symbols

    variables = tuple(free_symbols_f1.intersection(free_symbols_f2))
    pvariables = tuple(symbols(f'{symbol}_p') for symbol in variables)

    sum_terms = [(variable - pvariables[i])*diff(f1, variable).subs(variable, pvariables[i]) for i, variable in enumerate(variables)]
    proj = f2 - f1.subs(dict(zip(variables, pvariables))) - sum(sum_terms)

    return proj.expand(), variables, pvariables

def recursive_discriminant(expr, vars):
    '''
    Takes the discriminant over all variables in a given expression.

    Parameters:
        expr (sympy expression): The expression to take the discriminant of.

    Returns:
        discriminant (sympy expression): The discriminant of the expression.
    '''
    vars_list = list(vars)
    def recursive_discriminant_helper(expr, vars):
        # if there are no variables, return the expression
        if vars == []:
            return expr
        
        for var in vars_list:
            if expr.has(var):
                vars_list.remove(var)
                return quad_discriminant(recursive_discriminant(expr, vars_list), var)

            
    return recursive_discriminant_helper(expr, vars_list)

In [ ]:
# At each temperature, we need to find the equilibrium phases with the discriminant method and convex hull, and then plot the stable phases
pairs = list(combinations(phase_poly_dict.keys(), 2))
phase_values_dict = defaultdict(list)
for pair in pairs:
    phase1 = pair[0]
    phase2 = pair[1]

    proj, variables, pvariables = projection_function(phase_poly_dict[phase1], phase_poly_dict[phase2])
    discriminant = recursive_discriminant(proj, variables)
    solutions = [float(sol) for sol in sp.solve(discriminant, pvariables) if sol.is_real]
    phase_values_dict[phase1].extend(solutions)

    proj, variables, pvariables = projection_function(phase_poly_dict[phase2], phase_poly_dict[phase1])
    discriminant = recursive_discriminant(proj, variables)
    solutions = [float(sol) for sol in sp.solve(discriminant, pvariables) if sol.is_real]
    phase_values_dict[phase2].extend(solutions)

In [ ]:
points = np.empty((0, 2))
for phase in phase_values_dict:
    func = sp.lambdify(symbols('x'), phase_poly_dict[phase], 'numpy')
    x_vals = np.array(phase_values_dict[phase])
    y_vals = func(x_vals)

    points = np.concatenate((points, np.column_stack((x_vals, y_vals))), axis=0)

In [ ]:
from scipy.spatial import ConvexHull

def lower_convex_hull(points):
    '''
    Calculate the lower convex hull, assuming the last dimension represents energy.

    Parameters:
        points (array): Points in N-dimensional space, with the last dimension representing energy.

    Returns:
        lower_hull (array): Array of indices describing the points that form the lower convex hull.
    '''
    processing_points = points.copy()

    # Check if the projected points are collinear
    projected_points = processing_points[:, :-1]
    transformed_points = projected_points - projected_points[0]
    if np.linalg.matrix_rank(transformed_points) == 1:
        idx = np.argsort(np.linalg.norm(transformed_points, axis=1))
        bp = np.array([idx[0], idx[-1]])
        processing_points = processing_points[:, 1:]

    else:
        bp = ConvexHull(points).simplices.flatten()
    
    fake_points = processing_points[bp].copy()
    fake_points[:, -1] += 500000  # offset to create "upper" points
    processing_points = np.vstack((processing_points, fake_points))

    hull = ConvexHull(processing_points)
    simplices = hull.simplices

    mask = np.all(simplices < len(points), axis=1)
    lower_hull = simplices[mask]

    return lower_hull

In [ ]:
lower_convex_hull(points)

In [ ]:
conditions = { 
                v.T: 400,
                v.P: 101325,
                v.X("AL"): np.linspace(0, 1, 100)
            }

# Define equilibrium conditions properly
comps = ['AL', 'FE']
eq = equilibrium(db, comps, phases, conditions=conditions)  

print(eq)

In [ ]:
def refine_sampling(points, hull_points, alpha):
    """
    Refines sampling around hull points. For each segment on the hull, 
    we add more points in an interval determined by alpha.
    points: original sampled points (list of (x, y))
    hull_points: points that lie on the current hull (ordered)
    alpha: fraction of the distance along the hull edge to add more points
    Returns an augmented list of points.
    """
    new_points = list(points)  # start with existing points
    for i in range(len(hull_points)):
        p1 = np.array(hull_points[i])
        p2 = np.array(hull_points[(i + 1) % len(hull_points)])
        # Determine the distance and then the refinement spacing.
        dist = np.linalg.norm(p2 - p1)
        # We add points in the interval [p1, p2] with spacing = alpha*dist
        num_new = int(1 / alpha)  # number of new points to add in the interval
        # Generate new points between p1 and p2, excluding endpoints (they are already in hull_points)
        for j in range(1, num_new):
            t = j / num_new
            new_pt = (1 - t) * p1 + t * p2
            new_points.append(tuple(new_pt))
    return new_points

def adaptive_convex_hull(f, x_range, initial_resolution=0.1, alpha=0.1, max_iterations=5):
    """
    f: the function to sample, taking x as input and returning y.
    x_range: tuple (xmin, xmax)
    initial_resolution: step size for initial sampling.
    alpha: determines the refinement resolution around hull segments.
    max_iterations: maximum iterations to refine.
    """
    # Create initial sample points
    x_vals = np.arange(x_range[0], x_range[1] + initial_resolution, initial_resolution)
    points = [(x, f(x)) for x in x_vals]

    for i in range(max_iterations):
        hull = lower_convex_hull(points)
        # Optional: Plot the current hull and points
        xs, ys = zip(*points)
        hull_xs, hull_ys = zip(*hull)
        plt.figure(figsize=(8, 4))
        plt.plot(xs, ys, 'bo', markersize=3, label="Samples")
        plt.plot(hull_xs + (hull_xs[0],), hull_ys + (hull_ys[0],), 'r-', lw=2, label="Hull")
        plt.title(f"Iteration {i+1}")
        plt.legend()
        plt.show()

        # Refine the sampling around the current hull segments.
        points = refine_sampling(points, hull, alpha)
        
    return hull

def poly(x):
    return 0.5 * x**2 - 3 * x + 2

hull_points = adaptive_convex_hull(poly, x_range=(-5, 10), initial_resolution=0.5, alpha=0.1, max_iterations=4)

# Final hull visualization
xs, ys = zip(*[ (x, poly(x)) for x in np.linspace(-5, 10, 500) ])
plt.figure(figsize=(8, 4))
plt.plot(xs, ys, 'g--', label="True Function")
hull_xs, hull_ys = zip(*hull_points)
plt.plot(hull_xs + (hull_xs[0],), hull_ys + (hull_ys[0],), 'r-', lw=2, label="Adaptive Convex Hull")
plt.title("Final Convex Hull")
plt.legend()
plt.show()
